In [1]:
_author__ = 'Monkey'
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
# import matplotlib.pylab as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.metrics import make_scorer
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
# from matplotlib.colors import LogNorm
from sklearn.decomposition import PCA

from copy import deepcopy
import pdb

import csv
import time
start_time = time.time()


# import keras
from keras.layers.advanced_activations import PReLU
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.utils import np_utils

import xgboost as xgb

Using Theano backend.


In [2]:
train = pd.read_csv("train.csv")

In [3]:
def get_data(fn):
  data = []
  with open(fn) as f:
    reader = csv.DictReader(f)
    data = [row for row in reader]
  return data

label_train = get_data('train.csv')

In [4]:
labels = 'ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,EXTORTION,FAMILY OFFENSES,FORGERY/COUNTERFEITING,FRAUD,GAMBLING,KIDNAPPING,LARCENY/THEFT,LIQUOR LAWS,LOITERING,MISSING PERSON,NON-CRIMINAL,OTHER OFFENSES,PORNOGRAPHY/OBSCENE MAT,PROSTITUTION,RECOVERED VEHICLE,ROBBERY,RUNAWAY,SECONDARY CODES,SEX OFFENSES FORCIBLE,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS'.split(',')

label_fields = {'Category': lambda x: labels.index(x.replace(',', ''))}

def get_fields(data, fields):
  extracted = []
  for row in data:
    extract = []
    for field, f in sorted(fields.items()):
      info = f(row[field])
      if type(info) == list:
        extract.extend(info)
      else:
        extract.append(info)
    extracted.append(np.array(extract, dtype=np.float32))
  return extracted

# pdb.set_trace()

labels_int = np.array(get_fields(label_train, label_fields))
# pdb.set_trace()
# labels_int = np_utils.to_categorical(labels_int)

labels_cmp = np_utils.to_categorical(labels_int.flatten())

/Users/xiaoyiliu/anaconda/lib/python2.7/site-packages/keras/utils/np_utils.py:14: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Y = np.zeros((len(y), nb_classes))
/Users/xiaoyiliu/anaconda/lib/python2.7/site-packages/keras/utils/np_utils.py:16: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Y[i, y[i]] = 1.


In [5]:
xy_scaler = preprocessing.StandardScaler()
xy_scaler.fit(train[["X","Y"]])
train[["X","Y"]] = xy_scaler.transform(train[["X","Y"]])
train = train[abs(train["Y"]) < 100]
train.index = range(len(train))

In [35]:
addressFeats = list(train.Address.value_counts()[:50].index)

In [36]:
def extractAddressFeatures(x, addressFeats):
    if x not in addressFeats: return -1
    else: return addressFeats.index(x)

In [41]:
def parse_time (x):
    DD = datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
    time = DD.hour
    day = DD.day
    month = DD.month
    year = DD.year
    return time, day, month, year

def get_season (x):
    summer = 0
    fall = 0
    winter = 0
    spring = 0
    if (x in [5, 6, 7]):
        summer = 1
    if (x in [8, 9, 10]):
        fall = 1
    if (x in [11, 0, 1]):
        winter = 1
    if (x in [2, 3, 4]):
        spring = 1
    return summer, fall, winter, spring


# In[6]:

## replace address by log odds

def parse_data (df, logodds, logoddsPA):
    feature_list = df.columns.tolist()
    if "Descript" in feature_list:
        feature_list.remove("Descript")
    if "Resolution" in feature_list:
        feature_list.remove("Resolution")
    if "Category" in feature_list:
        feature_list.remove("Category")
    if "Id" in feature_list:
        feature_list.remove("Id")
    clean_Data = df[feature_list]
    clean_Data.index = range(len(df))
    print("Creating address features")
    address_features = clean_Data["Address"].apply(lambda x: logodds[x])
    # pdb.set_trace()
    address_features.columns = ["logodds" + str(x) for x in range(len(address_features.columns))]
    print("Parsing dates")
    clean_Data["Time"], clean_Data["Day"], clean_Data["Month"], clean_Data["Year"] = zip(*clean_Data["Dates"].apply(parse_time))
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    print("Creating one-hot variables")
    
    clean_Data['AddrImportance'] = clean_Data['Address'].apply(lambda x: extractAddressFeatures(x, addressFeats))
    dummy_importance = pd.get_dummies(clean_Data['AddrImportance'], prefix = 'Address_Importance')
    dummy_ranks_PD = pd.get_dummies(clean_Data['PdDistrict'], prefix = 'PD')
    dummy_ranks_DAY = pd.get_dummies(clean_Data["DayOfWeek"], prefix = 'DAY')
    clean_Data["IsInterection"] = clean_Data["Address"].apply(lambda x: 1 if "/" in x else 0)
    clean_Data["logoddsPA"] = clean_Data["Address"].apply(lambda x: logoddsPA[x])
    print("droping processed columns")
    clean_Data = clean_Data.drop("PdDistrict",axis = 1)
    clean_Data = clean_Data.drop("DayOfWeek",axis = 1)
    
    clean_Data = clean_Data.drop("Address",axis = 1)
    clean_Data = clean_Data.drop("AddrImportance",axis = 1)
    clean_Data = clean_Data.drop("Dates",axis = 1)
    feature_list = clean_Data.columns.tolist()
    print("joining one-hot features")
    features = clean_Data[feature_list].join(dummy_ranks_PD.ix[:,:]).join(dummy_ranks_DAY.ix[:,:]).join(address_features.ix[:,:]).join(dummy_importance.ix[:,:])
    print("creating new features")
    features["IsDup"] = pd.Series(features.duplicated()|features.duplicated(take_last = True)).apply(int)
    features["Awake"] = features["Time"].apply(lambda x: 1 if (x==0 or (x>=8 and x<=23)) else 0)
    features["Summer"], features["Fall"], features["Winter"], features["Spring"] = zip(*features["Month"].apply(get_season))
    if "Category" in df.columns:
        labels = df["Category"].astype(pd.core.categorical.Categorical)
    else:
        labels = None
    return features, labels

In [22]:
addresses = sorted(train["Address"].unique())
categories = sorted(train["Category"].unique())
C_counts = train.groupby(["Category"]).size()
A_C_counts = train.groupby(["Address","Category"]).size()
A_counts = train.groupby(["Address"]).size()
logodds = {}
logoddsPA = {}
MIN_CAT_COUNTS = 2
default_logodds = np.log(C_counts/len(train)) - np.log(1.0-C_counts/float(len(train)))

for addr in addresses:
    PA = A_counts[addr]/float(len(train))
    logoddsPA[addr] = np.log(PA)-np.log(1.-PA)
    logodds[addr] = deepcopy(default_logodds)
    for cat in A_C_counts[addr].keys():
        if (A_C_counts[addr][cat] > MIN_CAT_COUNTS) and A_C_counts[addr][cat] < A_counts[addr]:
            PA = A_C_counts[addr][cat]/float(A_counts[addr])
            logodds[addr][categories.index(cat)] = np.log(PA) - np.log(1.0-PA)
    logodds[addr] = pd.Series(logodds[addr])
    logodds[addr].index = range(len(categories))

In [42]:
features, labels = parse_data(train, logodds, logoddsPA)

Creating address features
Parsing dates
Creating one-hot variables
droping processed columns
joining one-hot features
creating new features


In [43]:
features.columns

Index([u'X', u'Y', u'Time', u'Day', u'Month', u'Year', u'IsInterection', u'logoddsPA', u'PD_BAYVIEW', u'PD_CENTRAL', u'PD_INGLESIDE', u'PD_MISSION', u'PD_NORTHERN', u'PD_PARK', u'PD_RICHMOND', u'PD_SOUTHERN', u'PD_TARAVAL', u'PD_TENDERLOIN', u'DAY_Friday', u'DAY_Monday', u'DAY_Saturday', u'DAY_Sunday', u'DAY_Thursday', u'DAY_Tuesday', u'DAY_Wednesday', u'logodds0', u'logodds1', u'logodds2', u'logodds3', u'logodds4', u'logodds5', u'logodds6', u'logodds7', u'logodds8', u'logodds9', u'logodds10', u'logodds11', u'logodds12', u'logodds13', u'logodds14', u'logodds15', u'logodds16', u'logodds17', u'logodds18', u'logodds19', u'logodds20', u'logodds21', u'logodds22', u'logodds23', u'logodds24', u'logodds25', u'logodds26', u'logodds27', u'logodds28', u'logodds29', u'logodds30', u'logodds31', u'logodds32', u'logodds33', u'logodds34', u'logodds35', u'logodds36', u'logodds37', u'logodds38', u'Address_Importance_-1', u'Address_Importance_0', u'Address_Importance_1', u'Address_Importance_2', u'Addres

In [44]:
collist = features.columns.tolist()
scaler = preprocessing.StandardScaler()
scaler.fit(features)
features[collist] = scaler.transform(features)

In [45]:
labels_binary = pd.get_dummies(labels)

In [46]:
test = pd.read_csv("test.csv")
test[["X","Y"]] = xy_scaler.transform(test[["X","Y"]])

test["X"] = test["X"].apply(lambda x: 0 if abs(x)>5 else x)
test["Y"] = test["Y"].apply(lambda y: 0 if abs(y)>5 else y)

new_addresses = sorted(test["Address"].unique())
new_A_counts=test.groupby("Address").size()
only_new = set(new_addresses + addresses) - set(addresses)
only_old = set(new_addresses + addresses) - set(new_addresses)
in_both = set(new_addresses).intersection(addresses)
for addr in only_new:
    PA = new_A_counts[addr]/float(len(test) + len(train))
    logoddsPA[addr] = np.log(PA) - np.log(1.-PA)
    logodds[addr] = deepcopy(default_logodds)
    logodds[addr].index = range(len(categories))
for addr in in_both:
    PA = (A_counts[addr] + new_A_counts[addr])/float(len(test)+len(train))
    logoddsPA[addr] = np.log(PA) - np.log(1.-PA)


features_sub, _ = parse_data(test, logodds, logoddsPA)

collist = features_sub.columns.tolist()

Creating address features
Parsing dates
Creating one-hot variables
droping processed columns
joining one-hot features
creating new features


In [47]:
features_sub[collist] = scaler.transform(features_sub[collist])

In [48]:
rnd = 57
param = {'max_depth': 8, 
         'eta': 0.1, 
         # next step: eta to 0.15
         'silent': 1, 
         'objective': 'multi:softprob',
         'eval_metric': "mlogloss",
         'min_child_weight': 1,
         'subsample': 0.7,
         'colsample_bytree': 0.7,
         'num_class': 39,
         'seed': rnd
        }

In [49]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
labelsInt = le.fit_transform(labels)

In [50]:
print max(labelsInt), min(labelsInt)

38 0


In [51]:
split = StratifiedShuffleSplit(labelsInt, 1, test_size = 0.15, random_state = 57)

In [52]:
for train_index, test_index in split:
    trainX = features.as_matrix()[train_index]
    trainY = labelsInt[train_index]
    cvX = features.as_matrix()[test_index]
    cvY = labelsInt[test_index]
    
    dtrain = xgb.DMatrix(trainX, label = trainY)
    dtest = xgb.DMatrix(cvX, label = cvY)

In [ ]:
watchlist  = [(dtrain,'train'), (dtest,'eval')]
numRound = 200

In [ ]:
xgbModel = xgb.train(param, dtrain, numRound, watchlist, early_stopping_rounds = 50)

In [ ]:
dtrainFull = xgb.DMatrix(features.as_matrix(), label = labelsInt)
watchlist  = [(dtrainFull,'train'), (dtrainFull,'eval')]
xgbModel = xgb.train(param, dtrainFull, numRound, watchlist, early_stopping_rounds = 50)

In [23]:
testPred = xgbModel.predict(xgb.DMatrix(features_sub.as_matrix()))

In [24]:
pred = pd.DataFrame(testPred, columns = sorted(labels.unique()))

In [26]:
import gzip
with gzip.GzipFile('SFCrimePredXGBoost.csv.gz',mode = 'w',compresslevel = 9) as gzfile: pred.to_csv(gzfile,index_label = "Id",na_rep = "0")